<a href="https://colab.research.google.com/github/Rami-RK/LangChain_EXploration/blob/main/Langchain_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Langchain_Agents**

### Objectives:
At the end of the experiment you will be able to understand & use :

1. built in  tools : wikipedia, llm-math
2. Python agent & tools
3. Google Search Engine Agent and tool-serpapi
4. how to build you own tool


In [ ]:
!pip install openai
!pip install langchain

In [ ]:
import openai
import os

In [ ]:
f = open('/content/openapi_key.txt')
api_key = f.read()
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key= os.getenv('OPENAI_API_KEY')

In [ ]:
llm_model = "gpt-3.5-turbo" # This is a chat model

## Built-in LangChain tools

In [ ]:
!pip install -U wikipedia

In [ ]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)
# we are going to use language model as the reasoning engine of an agent
# conneting to other sources data and computation so we want this reasoning engine as good as presice as possible
# so temperature is set to zero to get rid of any randomness.

In [ ]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)
# llm-math tool is a chain itself which uses a language model
# in conjuction with a calculater to do math problems
# "wikipedia" tool is an api  that connects with wikipedia allowing
# to run search querris against wikipedia and get back the results

we are going to initialize an agent with tools with language model and the agent type CHAT_ZERO_SHOT_REACT_DESCRIPTION, Note : **chat** this is an agent optimized to work with chat models, ***REACT** this is a prompting technique designed to get best reasoning performance out of language model

In [ ]:
agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True, # useful when language model output something that is not able to parse into an action input
    verbose = True) # Print steps what is going on

In [ ]:
agent("What is the 20% of 500?")



> Entering new AgentExecutor chain...
I can use the calculator tool to find the answer to this question.

Action:
```json
{
  "action": "Calculator",
  "action_input": "20% of 500"
}
```
Observation: Answer: 100.0
Thought:The answer is 100.0.
Final Answer: 100.0

> Finished chain.


{'input': 'What is the 20% of 500?', 'output': '100.0'}

See above result: enters into excecuter and --> thought --> action (a json blob containing 2 things : action(correspond to tool to use i.e. calculater) and action input(input to that tool))



#### **Wikipedia Example**

In [ ]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question)



> Entering new AgentExecutor chain...
Thought: I can use Wikipedia to find out what book Tom M. Mitchell wrote.
Action:
```json
{
  "action": "Wikipedia",
  "action_input": "Tom M. Mitchell"
}
```

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Tom Mitchell (Australian footballer)
Summary: Thomas Mitchell (born 31 May 1993) is a professional Australian rules footb

#### **Python Agent**

Co-pilot or chatgpt with code interpreter plugin -enabled, they are doing language model to writ code and executing that code. We are going to do same thing here. We are going to create a python agent and tool for that is `PythonREPLTool()` -a repl is a way to interact with code as jupyter notebook, so agent can execute the code with this repl and run and getback the result wich passed to agent to decide what to do next.

In [ ]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [ ]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")



> Entering new AgentExecutor chain...


I can use the `sorted()` function to sort the list of customers. I will need to provide a key function that specifies the sorting order based on last name and then first name.
Action: Python_REPL
Action Input: sorted([['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']], key=lambda x: (x[1], x[0]))
Observation: 
Thought:The customers have been sorted by last name and then first name.
Final Answer: [['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]"

#### View detailed outputs of the chains

In [ ]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")
langchain.debug=False

### **Google Search Engine**

Certain tools require API Keys, in this example, we use SERPER API, create a new account and access your API KEY HERE:
https://serpapi.com/users/welcome


In [ ]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32003 sha256=0010e14cdcb7bc2095c43738926fcd70505dfbe607397990200e6bd358e19646
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:
import os
ff = open('/content/serpapi_key.txt')
api_keyy = ff.read()
os.environ['SERPAPI_API_KEY'] = api_keyy

In [ ]:
tools = load_tools(["serpapi","llm-math"], llm=llm,)
# Optionally pass in via serpapi_api_key=#####

In [ ]:
dir(AgentType)

['CHAT_CONVERSATIONAL_REACT_DESCRIPTION',
 'CHAT_ZERO_SHOT_REACT_DESCRIPTION',
 'CONVERSATIONAL_REACT_DESCRIPTION',
 'OPENAI_FUNCTIONS',
 'OPENAI_MULTI_FUNCTIONS',
 'REACT_DOCSTORE',
 'SELF_ASK_WITH_SEARCH',
 'STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION',
 'ZERO_SHOT_REACT_DESCRIPTION',
 '__class__',
 '__doc__',
 '__members__',
 '__module__']

In [ ]:
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

In [ ]:
agent.run("When did recent G20 summit happen? What is that year number multiplied by 2?")



> Entering new AgentExecutor chain...
Question: When did the recent G20 summit happen? What is that year number multiplied by 2?
Thought: I can use a search engine to find the date of the recent G20 summit and then multiply the year number by 2 using a calculator.
Action:
```
{
  "action": "Search",
  "action_input": "recent G20 summit date"
}
```

Observation: Sat, Sep 9, 2023 – Sun, Sep 10, 2023
Thought:The recent G20 summit happened on September 9-10, 2023. Now I can calculate the year number multiplied by 2.
Action:
```
{
  "action": "Calculator",
  "action_input": "2023 * 2"
}
```

Observation: Answer: 4046
Thought:The recent G20 summit happened on September 9-10, 2023. The year number, 2023, multiplied by 2 is 4046.
Final Answer: 4046

> Finished chain.


'4046'

## Define your own tool

First we have to import the tool decorator which convert any function as tool that langchain can use

In [ ]:
#!pip install DateTime

In [ ]:
from langchain.agents import tool
from datetime import date

In [ ]:
import datetime

In [ ]:
@tool
def time(text: str) -> str:
    """Returns current date and time, use this for any \
    questions related to knowing current date and time. \
    The input should always be an empty string, \
    and this function will always return current  \
    date and time."""
    return str(datetime.datetime.now())

In [ ]:
agent= initialize_agent(
    tools + [time],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [ ]:
agent.run("what is the time current time?")